## Follow these steps to reload the vector index, currently running in DataStax

* **Gather Data** 
    * Download The [docs](https://github.com/fiddler-labs/fiddler/tree/main/docs) folder from the [Fiddler Repo](https://github.com/fiddler-labs/fiddler)
        * Clone or Make a pull request on the repo
        * Make sure this is grabbed from the branch corresponding to the current release (branch name should be `release/2x.x`)
        * Copy the Docs Folder, rename it to the correct release number (ex -v24.14) and put that folder in fiddler-chatbot/documentation_data
    * Copy the latest [quickstart notebooks](https://github.com/fiddler-labs/fiddler-examples)
        * Make sure to grab the notebook from the latest release
        * Place all the .ipynb files from for the lastest release in fiddler-chatbot/documentation_data/notebooks-ipynb
* **Run this notebook to generate the vector index feed for this verison**
    * Generate the markdown version of .ipynb files and add it to quickstart pages on with the script below
    * Crawl the our website for blog and other resources
    * You will need to add the caveats from last version the current version
    * Chunk the data to 750 token snippets
    * Finally we generate the vector_index_feed_24.x.csv that we will upload to our vector database

* **Last step:** Reloading the vector index table is done via the "loader_cassandra_vector_index.ipynb" notebook. Open that notebook and follow the instructions there

### Imports

In [ ]:
!pip -q install tiktoken
!pip -q install openai
!pip -q install scipy
!pip -q install ast
!pip -q install feedparser
!pip -q install bs4

In [ ]:
import os
import pandas as pd
import tiktoken
import openai 
import re
from scipy import spatial 
import ast
import feedparser
from bs4 import BeautifulSoup
import requests

### Set State

In [ ]:
EMBEDDING_MODEL = "text-embedding-ada-002"  # OpenAI's best embeddings as of Apr 2023
GPT_MODEL = "gpt-3.5-turbo"
release_num = '24.7'

In [ ]:
def num_tokens(text: str, model: str = GPT_MODEL) -> int:
    """Return the number of tokens in a string."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

def chunked_string(
    string: str,
    model: str = EMBEDDING_MODEL,
    max_tokens: int = 4000,
) -> str:
    """Truncate a string to a maximum number of tokens."""
    encoding = tiktoken.encoding_for_model(model)
    encoded_string = encoding.encode(string)
    chunked_string = [encoding.decode(encoded_string[i:i+max_tokens]) for i in range(0, len(encoded_string), max_tokens)]
    return chunked_string

## Append Notebook markdowns to quick starts

- Download [Quickstart guides](https://github.com/fiddler-labs/fiddler-examples) v24.x into *documentation_data/notebooks-ipynb* folder: 
- Running the NBConvert in the next step will update the md-notebooks folder with markdown version of all the quickstarts from the *notebooks-ipynb* directory 

In [ ]:
!jupyter nbconvert --output-dir='documentation_data/md-notebooks' documentation_data/notebooks-ipynb/*.ipynb --to markdown

The cell will take the markdown version of the notebooks and append it to the quickstart pages in the documentation directory 

In [ ]:
for root, dirs, files in os.walk(f'/Users/anushrav/Projects/fiddler-chatbot/documentation_data/{release_num}/QuickStart_Notebooks'): # make sure this path points to your .ipynb notebooks
    for name in files:
        path = os.path.join(root, name)
        if path[-3:] == '.md':
            with open(path,'r') as f:
                file_str = f.read()    
            print(file_str)
            ipynb_links = re.search(r'\bFiddler_Quickstart_\w+', file_str)
            print(ipynb_links)
            if ipynb_links:
                with open('/Users/anushrav/Projects/fiddler-chatbot/documentation_data/md-notebooks/'+ipynb_links.group()+'.md') as l: 
                    QS = l.read()
                with open(path, 'a') as f:
                    f.write(QS)
                print(ipynb_links.group())

## Creating list of chunked_docs from downloaded documentation 

In [ ]:
#change the path to where your downloaded folder is and choose the version of the docs you want to process
chunked_doc = []
for root, dirs, files in os.walk(f'documentation_data/{release_num}'):
    for name in files:
        path = os.path.join(root, name)
        if path[-3:] == '.md':
            with open(path,'r') as f:
                file_str = f.read()
                chunked_doc.append(file_str)
                
len(chunked_doc)

In [ ]:
#find and remove hidden pages
pattern = r'hidden:\s*(\w+)'

for doc in chunked_doc:
    match = re.search(pattern, doc)
    if match and match.group(1) == "true":
        chunked_doc.remove(doc)
        
len(chunked_doc)

### Crawl the blog and resources content and append it to chunked_doc list

In [ ]:
def crawl_rss_feed(rss_url):
    # Parse the RSS feed
    feed = feedparser.parse(rss_url)

    print("Number of Blogs:", len(feed.entries))
    
    # Iterate through the entries in the feed
    for entry in feed.entries:

        # Get the URL of the blog article
        article_url = entry.link

        # Fetch the content of the article
        response = requests.get(article_url)
        html_content = response.content.decode('utf-8', 'ignore')

        # Use BeautifulSoup to parse the HTML and extract the body
        soup = BeautifulSoup(html_content, 'html.parser')
        div_content = soup.find('div', class_='blog-post_content-wrapper')  # You may need to adjust this based on the HTML structure

        # Print or manipulate the content of the div
        if div_content:
            print("Title:", entry.title)
            print("Link:", entry.link)
            itemtext=''
            for item in div_content.select('p'):
                itemtext+=item.text + ' '
            chunked_doc.append("BlogLink:" + entry.link + " Content: " + itemtext)
        else:
            print("Div not found.")

# Replace 'your_rss_feed_url' with the actual RSS feed URL
rss_feed_url = 'https://www.fiddler.ai/blog/rss.xml'
crawl_rss_feed(rss_feed_url)

In [ ]:
def crawl_rss_feed(rss_url):
    # Parse the RSS feed
    feed = feedparser.parse(rss_url)

    print("Number of Resources:", len(feed.entries))
    
    # Iterate through the entries in the feed
    for entry in feed.entries:

        # Get the URL of the blog article
        article_url = entry.link

        # Fetch the content of the article
        response = requests.get(article_url)
        html_content = response.content.decode('utf-8', 'ignore')

        # Use BeautifulSoup to parse the HTML and extract the body
        soup = BeautifulSoup(html_content, 'html.parser')

        
        div_content = soup.find('div', class_='resources-copy')  # You may need to adjust this based on the HTML structure

        # Print or manipulate the content of the div
        if div_content:
            print("Title:", entry.title)
            print("Link:", entry.link)
            itemtext=''
            for item in div_content.select('p'):
                itemtext+=item.text + ' '
            chunked_doc.append("ResourceLink:" + entry.link + " Content: " + itemtext)
        else:
            print("Div not found.")

# Replace 'your_rss_feed_url' with the actual RSS feed URL
rss_feed_url = 'http://www.fiddler.ai/resources/rss.xml'
crawl_rss_feed(rss_feed_url)

In [ ]:
len(chunked_doc)

In [ ]:
#we will append page slugs or blog links to every chunk
slug_pattern = r'slug:\s*"(.*?)"'
blog_pattern = r'BlogLink:https://www\.fiddler\.ai/blog/([\w-]+)'
resource_pattern = r'ResourceLink:https://www\.fiddler\.ai/resources/([\w-]+)'

In [ ]:
#chunking docs to 750 tokens

token_lim_doc = []
for doc in chunked_doc:
    if num_tokens(doc) > 750:
        chunked_list = chunked_string(doc, max_tokens=750)
        
        # see if a doc slug or a blog link is detected
        slug = re.search(slug_pattern, chunked_list[0])
        blog = re.search(blog_pattern, chunked_list[0])

        if slug:
            chunked_doc_slug = slug.group(0)
            for i in range(1, len(chunked_list)):
                chunked_list[i] = chunked_doc_slug + ' ' + chunked_list[i]
        
        if blog:
            chunked_doc_blog = blog.group(0)
            for i in range(1, len(chunked_list)):
                chunked_list[i] = chunked_doc_blog + ' ' + chunked_list[i]
        
        token_lim_doc += chunked_list
    else:
        token_lim_doc.append(doc)

In [ ]:
#Read in old caveats from previous version and output them to a new file
old_df = pd.read_csv('documentation_data/v24.5/caveats.csv')
caveats_df = old_df[~old_df['text'].str.contains('slug', case=False)]
#caveats_df = caveats_df.drop(columns=['embedding'])
caveats_df.to_csv(f'documentation_data/v{release_num}/caveats.csv', index=False)

caveats_df.loc[len(caveats_df.index)] = ['LLM means large language model.  A large language model (LLM) is a type of artificial intelligence (AI) algorithm that uses deep learning techniques and massively large data sets to understand, summarize, generate and predict new content.']
caveats_df.loc[len(caveats_df.index)] = ['The term generative AI, or GenAI, also is closely connected with LLMs, which are, in fact, a type of generative AI that has been specifically architected to help generate text-based content.']
caveats_df.loc[len(caveats_df.index)] = ['FM, or FMs, means Foundation Models.  Foundation Models are the same as large language models.']

caveats_df

In [ ]:
#embeddings = generate_embeddings(token_lim_doc)
#df = pd.DataFrame({"text": chunked_doc, "embedding": embeddings})
df = pd.DataFrame({"text": token_lim_doc})
df = pd.concat([df,caveats_df], ignore_index=True)
df.to_csv(f'documentation_data/vector_index_feed_{release_num}.csv', index=False)
df

### You are done! Please check the **vector_index_feed_{release_num}.csv** file and navigate to loader_casandra_vector_index.ipynb to upload these snippets to our vector database.